In [1]:
library(magrittr)
library(MASS)

## Refactoring

Refactoring code is a very important concept in programming. To "refactor" means to change the implementation of some piece of code, without changing the functionality. 

If your goal with writing code is simply to "get something to work" (once), then refactoring is not useful. If, however, your goal is to write elegant code which can be used reliable, extended, and modified, then you will always need to refactor! 

In [ ]:
# We will refactor the code from the "Functions in R" lecture. 

# Using the ideas from the slides, make "generate_data" a polymorphic function.
# It should operate on two classes: "univariate_params" and "multivariate_params"
# and two separate "methods" should be implemented for each of the classes 

In [ ]:
# The implementation of the class is up to you! So write the params class to
# match your design. 

# beta should be c(1,2,1)
# mean of X's should be c(0,0,0)
# sd of X's should be c(.2,.5,.3)
# noise sd should be .5

generate_data <- function (params) {
    UseMethod("generate_data")
}

generate_data.multivariate_params <- function(params) {
        
       N <- params$N
       beta <- params$beta
       params1 <- cbind(params$means, params$sd)
       sd <- params$noise 
    
       x <- apply(params1, 1, function(x) rnorm(N, mean = x[1], sd = x[2]))
       x <- cbind(rep(1, N), x)
       eps <- rnorm(N, 0, sd)
       y <- x%*%beta + eps
       list(x = x, y = y)
       
}
                  
generate_data.univariate_params <- function (N, beta, sd) {
    
       N <- params$N
       beta <- params$beta
       sd <- params$noise
    
       x <- rnorm(N, 0, 1)
       eps <- rnorm(N, 0, sd)
       y <- beta*x + eps
       list(x = x, y = y)
}
                  
calc_coef <- function(y,x) {
  coef <- solve(t(x)%*%x, t(x)%*%y)
  coef
}
                  
calc_se <- function(y, x, coef) {
      
      eps <- y - x %*% coef
      sigma2 <- (t(eps) %*% eps) / (nrow(x) - ncol(x))
      varmatrix <- c(sigma2) * solve(t(x) %*% x)
      se<- sqrt(diag(varmatrix))
}

run_regression <- function(y, x) {
    coef <- calc_coef(y, x)
    se <- calc_se(y, x, coef)
    model <- list(coef=coef, se=se)
    class(model) <- "simple_model"
    model
}
                  
                
 eval_model <- function(model, beta, conf = 1.96) {
    UseMethod("eval_model")
}
                
                  
eval_model.simple_model <- function(model, beta, conf = 1.96) {
    coef <- model$coef
    se <- model$se
    up <- coef + se*conf
    down <- coef - se*conf
    beta > down & beta < up
}

                  
simulate <- function(N, beta, params, sd) {
       d <- generate_data(N, beta, params, sd)
       m <- run_regression(d$y, d$x)
       eval_model(m$coef, m$se, beta)
  }

                  
avg_simulations <- function(M, N, beta, params, sd) {
       inside <- sapply(1:M, function (x) {
           simulate(N, beta, params, sd)
})
       sum(inside) / M
   }
                  




params <- list(N = N, beta = beta, means = params1, sd = sd, noise = noise)
class(params) <- c("univariate", "multivariate_params") 

In [ ]:
a <- avg_simulations(1000, 20, params)
stopifnot(round(a, 1) == .9)

In [ ]:
a <- avg_simulations(1000, 500, params)
stopifnot(a > .92)

In [ ]:
# BONUS

# The calc_coef and calc_se functions, in their multivariate form, are 
# naturally a generalization, and work for univariate data do. 

# But let's pretend they're not. 

# Write them as polymorphic functions, such that when you simulation gets called
# with univariate data, the (from-the-slides) univariate formula gets called, 
# and the (from-your-exersizes) multivariate formula gets called when 
# multivariate parameters are given to the simulations. 

# Note: be creative in your design! Add more classes if you think it helps!
# Think about which functions you want to be class-agnostic and which to be
# class-aware! 


